In [2]:
# Install the package
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [3]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from bert_score import BERTScorer

# Initialize the BERTScorer object
scorer = BERTScorer(lang='en')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
df = pd.read_csv('text_evaluation_data.csv', sep=";")
df.head()

,DETECTED_TEXT,GROUND_TRUTH
0,1.12,!?
1,A thousand curses. prisoner is gone.,A thousand curses! Prisoner is gone! ...
2,My poor snowy!. Did those beasts flatten him?.,My poor Snowy!… Did those beasts flatten him?
3,here we are. Snowy!,Here we are Snowy! ... Chicago!
4,Beaten! He's defeated again! me unless,Beaten! He's defeated me again! ... Unless ...


In [5]:
df.loc[pd.isna(df['DETECTED_TEXT']) == 1, "DETECTED_TEXT"] = ""

In [6]:
precisions = []
recalls = []
f1s = []

for i in range(len(df)):
    hypo = df["DETECTED_TEXT"][i]
    ref = df["GROUND_TRUTH"][i]

    P, R, F1 = scorer.score([ref], [hypo])

    precisions.append(P.numpy())
    recalls.append(R.numpy())
    f1s.append(F1.numpy())

In [7]:
df_clean = df.copy()

In [8]:
from string import digits

In [9]:
for i, text in enumerate(df_clean["DETECTED_TEXT"]):
    gt = df_clean["GROUND_TRUTH"][i].lower()
    gt = gt.replace(".", "").replace(",", "").replace("'", "").replace("!", "").replace("?", "").replace('"', '')
    gt = gt.replace("\\", "").replace("-", " ").replace(":", "").replace(".", "")
    remove_digits = str.maketrans('', '', digits)
    gt = gt.translate(remove_digits)
    df_clean["GROUND_TRUTH"][i] = gt

    text = text.lower()
    text = text.replace(".", "").replace(",", "").replace("'", "").replace("!", "").replace("?", "").replace('"', '')
    text = text.replace("\\", "").replace("-", " ").replace(":", "").replace(".", "")
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    df_clean["DETECTED_TEXT"][i] = text

In [10]:
precisions_clean = []
recalls_clean = []
f1s_clean = []

for i in range(len(df_clean)):
    hypo = df_clean["DETECTED_TEXT"][i]
    ref = df_clean["GROUND_TRUTH"][i]

    if hypo == ref:
      P = [1]
      R = [1]
      F1 = [1]

      precisions_clean.append(np.array(P))
      recalls_clean.append(np.array(R))
      f1s_clean.append(np.array(F1))

    else:
      P, R, F1 = scorer.score([ref], [hypo])

      precisions_clean.append(P.numpy())
      recalls_clean.append(R.numpy())
      f1s_clean.append(F1.numpy())

In [11]:
print(np.mean(precisions), np.mean(recalls), np.mean(f1s))

0.8664652 0.8797923 0.87291604


In [12]:
print(np.mean(precisions_clean), np.mean(recalls_clean), np.mean(f1s_clean))

0.9064447883030643 0.9120530210759329 0.9090663249725881


In [33]:
import numpy as np

In [5]:
# Define the reference and candidate sentences
reference = ["I have a daughter."]
candidate = ["My daughter is 12 years old."]

# Compute the BERTScore
P, R, F1 = scorer.score(candidate, reference)

# Print the scores
print("Precision: {:.2f}, Recall: {:.2f}, F1: {:.2f}".format(P.item(), R.item(), F1.item()))


Precision: 0.92, Recall: 0.95, F1: 0.93


In [ ]:
# Define the reference and candidate sentences
reference = ["The cat sat on the mat."]
candidate = ["A cat is sitting on a mat."]

# Compute the BERTScore
P, R, F1 = scorer.score(candidate, reference)

# Print the scores
print("Precision: {:.2f}, Recall: {:.2f}, F1: {:.2f}".format(P.item(), R.item(), F1.item()))

Precision: 0.95, Recall: 0.96, F1: 0.96
